<a href="https://colab.research.google.com/github/AnnJungChan/TECHLOSS_SIGNLAB/blob/main/WAIT/%EC%95%88%EC%A4%91%EC%B0%AC/2022-03-23/ECU_LINEAR_REGRESSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECU 테이블의 C1~C8과 CURRENT사이의 관계

- C1~C8을 모두 더하면 CURRENT가 되는지 파악한다.
- 다 더했을 때, CURRENT가 되지 않으면, 이유가 무엇인지 파악한다.
  - C1~C8까지의 합과 CURRENT간의 선형관계가 CONTROL이 달라짐에 따라서 바뀌는지 판단한다.

## 패키지 로딩

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 데이터 로딩

In [2]:
ECU = pd.read_csv('/content/gdrive/MyDrive/DB/ECU.csv')

In [3]:
ECU=pd.concat([ECU.iloc[:,0:4],ECU.iloc[:,12:22], ECU.iloc[:,23:26]], axis = 1)

C1~C8과 CURRENT사이의 관계를 규명하기 위해 필요한 정보들만 불러옴

In [4]:
ECU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620535 entries, 0 to 620534
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SHIP_ID          620535 non-null  object 
 1   D_INDEX          620535 non-null  int64  
 2   DEVICE_ID        620535 non-null  int64  
 3   CONTROL          620535 non-null  int64  
 4   C1               620535 non-null  int64  
 5   C2               620535 non-null  int64  
 6   C3               620535 non-null  int64  
 7   C4               620535 non-null  int64  
 8   C5               620535 non-null  int64  
 9   C6               620535 non-null  int64  
 10  C7               620535 non-null  int64  
 11  C8               620535 non-null  int64  
 12  TEMP             620535 non-null  float64
 13  CURRENT          620535 non-null  int64  
 14  CONTROL_CURRENT  620535 non-null  int64  
 15  SYSTEM           620535 non-null  int64  
 16  SECTION          620535 non-null  int6

# C1~C8의 합과 CURRENT 사이의 관계

In [5]:
ECU['SUM'] = ECU.iloc[:,4:12].sum(1)

In [6]:
ECU_SAME = ECU.loc[ECU['SUM'] == ECU['CURRENT']]

In [7]:
ECU_SAME

,SHIP_ID,D_INDEX,DEVICE_ID,CONTROL,C1,C2,C3,C4,C5,C6,C7,C8,TEMP,CURRENT,CONTROL_CURRENT,SYSTEM,SECTION,SUM
1,T140113-00102,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
2,T140113-00103,1,1,34,736,738,738,739,738,736,736,736,3.9,5897,5901,0,0,5897
4,T140128-00703,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
5,T140703-00101,1,1,0,936,943,937,0,0,0,0,0,0.0,2816,0,0,0,2816
7,T141027-00501,1,1,34,498,498,498,498,498,498,498,498,3.5,3984,0,0,0,3984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620530,T20200113001,19680,9,34,718,717,717,717,717,717,718,717,0.0,5738,5745,1,1,5738
620531,T20200113001,19683,9,34,846,848,849,847,850,848,850,831,0.0,6769,8000,1,1,6769
620532,T20200113001,19686,9,34,989,989,989,990,989,989,989,989,0.0,7913,7927,1,1,7913
620533,T20200113001,19689,9,34,998,998,997,997,997,997,997,998,0.0,7979,7991,1,1,7979


217244개의 행들이 C1~C8의 합과 CURRENT의 값이 같다는 사실을 알 수 있다.

In [8]:
ECU_DIFF = ECU.loc[ECU['SUM'] != ECU['CURRENT']]

In [9]:
ECU_DIFF

,SHIP_ID,D_INDEX,DEVICE_ID,CONTROL,C1,C2,C3,C4,C5,C6,C7,C8,TEMP,CURRENT,CONTROL_CURRENT,SYSTEM,SECTION,SUM
0,T140113-00101,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,64,-1,0,0,0
3,T140113-00104,1,1,34,658,658,657,661,660,4,658,657,3.2,4609,5306,0,0,4613
6,T140708-00101,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,312,-1,0,0,0
17,T20191016008,1,1,-1,119,120,0,0,0,0,0,0,-9999.0,0,-1,0,0,239
29,T170102-00101,2,1,-1,0,0,0,0,0,0,0,0,-9999.0,1472,-1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620084,T20200113001,15531,9,34,732,732,732,732,8,732,733,732,0.0,5125,5871,1,1,5133
620085,T20200113001,15534,9,34,735,735,734,735,8,735,734,733,0.0,5141,5871,1,1,5149
620086,T20200113001,15537,9,34,731,731,731,731,8,731,731,731,0.0,5117,5856,1,1,5125
620087,T20200113001,15540,9,34,738,738,738,738,8,738,738,738,0.0,5166,5919,1,1,5174


403291개의 행들이 C1~C8의 합과 CURRENT의 값이 다르다는 사실을 알 수 있다.

# C1~C8까지의 합과 CURRENT의 값이 같을 때

- ECU_SAME 테이블은 C1~C8까지의 합과 CURRENT의 값이 같을 때를 모두 모아놓은 데이터 셋이다.

- CONTROL, TEMP가 어떻게 형성되는지 파악할 것이다.

- 같을 때의 SHIP_ID/SECTION/DEVICE_ID를 파악할 것이다.

**CONTROL의 의미**



```
- -1 
  - 부호 절대값: 10000001
  - 1의 보수 : 11111110
  - 2의 보수 : 11111111
    (#default or null 로 해석?)
- 34 : 00100010
- 0 : 00000000
- 33 : 00100001
- 160 : 10100000
- 32 : 00100000
- 1 : 00000001
- 40 : 00101000
```

In [10]:
ECU_SAME['CONTROL'].value_counts()

 34     94604
-1      78521
 0      44055
 33        55
 160        3
 32         3
 1          2
 40         1
Name: CONTROL, dtype: int64

## CONTROL이 -1일 때

### TEMP의 분포

In [11]:
ECU_SAME_m1=ECU_SAME.loc[ECU_SAME['CONTROL'] == -1,:]
ECU_SAME_m1

,SHIP_ID,D_INDEX,DEVICE_ID,CONTROL,C1,C2,C3,C4,C5,C6,C7,C8,TEMP,CURRENT,CONTROL_CURRENT,SYSTEM,SECTION,SUM
1,T140113-00102,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
4,T140128-00703,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
8,T160127-00202,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
9,T170102-00101,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
12,T20181218003,1,1,-1,0,0,0,0,0,0,0,0,-9999.0,0,-1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617216,T20190115004,6924,8,-1,869,869,869,870,869,869,870,869,-9999.0,6954,-1,0,0,6954
617217,T20190115004,6928,8,-1,868,868,868,868,868,868,868,869,-9999.0,6945,-1,0,0,6945
617218,T20190115004,6932,8,-1,872,872,872,872,872,872,872,873,-9999.0,6977,-1,0,0,6977
617221,T20190115004,6936,8,-1,871,870,871,870,871,870,871,870,-9999.0,6964,-1,0,0,6964


In [14]:
ECU_SAME_m1['TEMP'].unique()

array([-9999.])

control = -1 일 때, 온도가 모두 -9999인 것이 확인되었다.

### CONTROL_CURRENT

- CURRENT의 목표값을 설정하는 곳

- CURRENT의 목표값과 실제 CURRENT가 일치하면 가장 최적이고, 비슷해도 최적이라고 할 수 있다. 하지만, 너무 엇나가버리면, 목표치를 달성하지 않았다는 의미로 받아들여도 된다.

In [17]:
ECU_SAME_m1['CONTROL_CURRENT'].unique()

array([-1])

CONTROL_CURRENT가 모두 -1로써, 디폴트 값으로 기록되고 있다. 이 경우 CURRENT가 0이어야 하는 것으로 "*추측*"된다.

In [19]:
ECU_SAME_m1[ECU_SAME_m1['SUM'] != 0]

,SHIP_ID,D_INDEX,DEVICE_ID,CONTROL,C1,C2,C3,C4,C5,C6,C7,C8,TEMP,CURRENT,CONTROL_CURRENT,SYSTEM,SECTION,SUM
38,T20181218003,3,1,-1,900,900,0,0,0,0,0,0,-9999.0,1800,-1,0,0,1800
39,T20190324001,3,1,-1,751,751,752,751,752,751,752,751,-9999.0,6011,-1,0,0,6011
59,T20201210005,4,1,-1,877,877,877,877,877,877,877,877,-9999.0,7016,-1,0,0,7016
60,T20201210006,4,1,-1,546,545,546,545,546,545,545,546,-9999.0,4364,-1,0,0,4364
69,T20181218003,5,1,-1,898,899,0,0,0,0,0,0,-9999.0,1797,-1,0,0,1797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617216,T20190115004,6924,8,-1,869,869,869,870,869,869,870,869,-9999.0,6954,-1,0,0,6954
617217,T20190115004,6928,8,-1,868,868,868,868,868,868,868,869,-9999.0,6945,-1,0,0,6945
617218,T20190115004,6932,8,-1,872,872,872,872,872,872,872,873,-9999.0,6977,-1,0,0,6977
617221,T20190115004,6936,8,-1,871,870,871,870,871,870,871,870,-9999.0,6964,-1,0,0,6964


### CONTROL의 작동

- CONTROL = -1이면 2의 보수로써의 이진법으로 나타낼 시 11111111이 됨을 알 수 있다.

- 즉, C1~C8이 모두 작동했을 때, CONTROL값이 -1임을 알 수 있다.

- SUM = 0이면, SUM과 CURRENT가 모두 같을 경우, C1~C8이 모두 작동했다고 간주하고, 정상이라고 판단한다. 즉, SUM이 0이 아닐때, 모든 C1~C8값이 정상적으로 들어왔는지 판단한다.

In [16]:
ECU_SAME_m1ECU_SAME_m1['SUM'] != 0

1         False
4         False
8         False
9         False
12        False
          ...  
617216     True
617217     True
617218     True
617221     True
617228     True
Name: SUM, Length: 78521, dtype: bool

# control 값이 c1~c8과 current 사이의 선형관계를 해치는가?

## CONTROL = 0 일때

In [13]:
CONTROL_CURRENT=pd.concat([CONTROL, CURRENT],axis = 1)

NameError: ignored

In [ ]:
CONTROL_CURRENT_0 = CONTROL_CURRENT[CONTROL_CURRENT['CONTROL'] == 0]

In [ ]:
X = np.array(CONTROL_CURRENT_0['CURRENT']).reshape(-1,1)
Y = np.array(CONTROL_CURRENT_0['C_SUM']).reshape(-1,1)

In [ ]:
control_0_result = LinearRegression()
control_0_result.fit(X,Y)

In [ ]:
control_0_result.coef_

In [ ]:
control_0_result.intercept_

In [ ]:
%matplotlib inline
plt.figure(figsize = (20,10))
plt.plot(X,Y,'o')
plt.plot(X,control_0_result.predict(X),'red')
plt.show()

## CONTROL = -1 일때

In [ ]:
CONTROL_CURRENT=pd.concat([CONTROL, CURRENT],axis = 1)

In [ ]:
CONTROL_CURRENT_minus_1 = CONTROL_CURRENT[CONTROL_CURRENT['CONTROL'] == -1]

In [ ]:
X = np.array(CONTROL_CURRENT_minus_1['CURRENT']).reshape(-1,1)
Y = np.array(CONTROL_CURRENT_minus_1['C_SUM']).reshape(-1,1)

In [ ]:
control_minus_1_result = LinearRegression()
control_minus_1_result.fit(X,Y)

In [ ]:
control_minus_1_result.coef_

In [ ]:
control_minus_1_result.intercept_

In [ ]:
%matplotlib inline
plt.plot(X,Y,'o')
plt.plot(X,control_minus_1_result.predict(X))
plt.show()

## CONTROL = 34 일때

In [ ]:
CONTROL_CURRENT=pd.concat([CONTROL, CURRENT],axis = 1)

In [ ]:
CONTROL_CURRENT_34 = CONTROL_CURRENT[CONTROL_CURRENT['CONTROL'] == 34]

In [ ]:
X = np.array(CONTROL_CURRENT_34['CURRENT']).reshape(-1,1)
Y = np.array(CONTROL_CURRENT_34['C_SUM']).reshape(-1,1)

In [ ]:
control_34_result = LinearRegression()
control_34_result.fit(X,Y)

In [ ]:
control_34_result.coef_

In [ ]:
control_34_result.intercept_

In [ ]:
%matplotlib inline
plt.plot(X,Y,'o')
plt.plot(X,control_0_result.predict(X),'red')
plt.show()